In [1]:
import numpy as np
import pandas as pd
import glob
import pickle
import scipy.io
import librosa
import matplotlib.pyplot as plt
from scipy.fft import rfft, rfftfreq
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import keras
from keras.models import model_from_json
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from keras.callbacks import EarlyStopping
from keras import Sequential
from scipy.signal import periodogram
from scipy.signal import firwin, lfilter

# Train on GPU

In [2]:
tf.config.set_visible_devices([], 'GPU')

In [3]:
frame_length = 600
overlap = 300
cutoff_freq = [1, 15]
num_taps = 50
h = firwin(num_taps, cutoff_freq, fs=100, pass_zero=False)
n_components = 7

In [4]:
root = "/Users/albk/Documents/Code/Real_Projects/MicroSleep/Datas/Track#2 Microsleep detection from single-channel EEG/Training set"

for n in range(1, 51):
  if n < 10:
    with open(root + f"/Data_Sample0{n}.mat", "rb") as g:
      globals()[f"df{n}"] = scipy.io.loadmat(g)
  else:
    with open(root + f"/Data_Sample{n}.mat", "rb") as r:
      globals()[f"df{n}"] = scipy.io.loadmat(r)

In [5]:
# training Data sort
drowsy_freq = []
awake_freq = []
drowsy_inds = {}
awake_inds = {}

# this loop iterates over labels and get index of each 0 and 1 class to awake_inds and drowsy_inds lists 
for d in range(1, 51):
  drowsy_inds[f"df{d}"] = []
  awake_inds[f"df{d}"] = []

  for ind, label in enumerate(globals()[f"df{d}"]["epo"][0][0][1][0]):
    if label == 1:
      drowsy_inds[f"df{d}"].append(ind)
    elif label == 0:
      awake_inds[f"df{d}"].append(ind)

# after sorting indexes of classes, we'll sort and save each 3000-sample window by their classes to drowsy and awake lists
  for lb1, dr in enumerate(globals()[f"df{d}"]["epo"][0][0][0]):
    if lb1 in drowsy_inds[f"df{d}"]:
      # dr = lfilter(h, 1, dr)
      windows = librosa.util.frame(dr, frame_length=frame_length, hop_length=overlap).transpose(1, 0)
      han_windows = np.multiply(windows, np.hanning(frame_length))
      feats = []
      for win in han_windows:
        feats.append(np.array(np.abs(np.array(rfft(win)[10:180])).tolist()))
        # feats.append(mfccs_feats(win, 100))

      drowsy_freq.append(feats)

    if len(awake_freq) != 6482:
      if lb1 in awake_inds[f"df{d}"]:
        # dr = lfilter(h, 1, dr)
        windows = librosa.util.frame(dr, frame_length=frame_length, hop_length=overlap).transpose(1, 0)
        han_windows = np.multiply(windows, np.hanning(frame_length))
        feats = []        
        for win in han_windows:
          feats.append(np.array(np.abs(np.array(rfft(win)[10:180])).tolist()))
          # feats.append(mfccs_feats(win, 100))
        
        awake_freq.append(feats)

In [6]:
pca = PCA(n_components=n_components).fit(np.vstack([awake_freq, drowsy_freq]).reshape((len(awake_freq) + len(drowsy_freq)) * len(awake_freq[0]), len(awake_freq[0][0])))

awake_freq = pca.transform(np.real(np.array(awake_freq).reshape(len(awake_freq) * len(awake_freq[0]), len(awake_freq[0][0])))).reshape(len(awake_freq), len(awake_freq[0]), n_components).tolist()
drowsy_freq = pca.transform(np.real(np.array(drowsy_freq).reshape(len(drowsy_freq) * len(drowsy_freq[0]), len(drowsy_freq[0][0])))).reshape(len(drowsy_freq), len(drowsy_freq[0]), n_components).tolist()

scaler = StandardScaler().fit(np.real(np.array(awake_freq + drowsy_freq).reshape((len(awake_freq) + len(drowsy_freq)) * len(awake_freq[0]), len(awake_freq[0][0]))))

awake_freq = scaler.transform(np.real(np.array(awake_freq).reshape(len(awake_freq) * len(awake_freq[0]), len(awake_freq[0][0])))).reshape(len(awake_freq), len(awake_freq[0]), len(awake_freq[0][0]))
drowsy_freq = scaler.transform(np.real(np.array(drowsy_freq).reshape(len(drowsy_freq) * len(drowsy_freq[0]), len(drowsy_freq[0][0])))).reshape(len(drowsy_freq), len(drowsy_freq[0]), len(drowsy_freq[0][0]))

In [7]:
import random

d_label = [1] * (drowsy_freq.shape[0])
a_label = [0] * (awake_freq.shape[0])

# Zip the features and labels
data_d = list(zip(drowsy_freq, d_label))
data_a = list(zip(awake_freq, a_label))
data = data_a + data_d

# Shuffle the data
random.shuffle(data)

# Unzip the data
features, labels = zip(*data)

features = np.array(features, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)

In [8]:
# validation_set loading
v_root = "/Users/albk/Documents/Code/Real_Projects/MicroSleep/Datas/Track#2 Microsleep detection from single-channel EEG/Validation set"
for n in range(1, 11):
  if n < 10:
    with open(v_root + f"/Data_Sample0{n}.mat", "rb") as g:
      globals()[f"df{n}"] = scipy.io.loadmat(g)
  else:
    with open(v_root + f"/Data_Sample{n}.mat", "rb") as r:
      globals()[f"df{n}"] = scipy.io.loadmat(r)

In [9]:
# validation Data sort
v_drowsy_freq = []
v_awake_freq = []
drowsy_inds = {}
awake_inds = {}

# this loop iterates over labels and get index of each 0 and 1 class to awake_inds and drowsy_inds lists 
for d in range(1, 11):
  drowsy_inds[f"df{d}"] = []
  awake_inds[f"df{d}"] = []

  for ind, label in enumerate(globals()[f"df{d}"]["epo"][0][0][2][0]):
    if label == 1:
      drowsy_inds[f"df{d}"].append(ind)
    elif label == 0:
      awake_inds[f"df{d}"].append(ind)

# after sorting indexes of classes, we'll sort and save each 3000-sample window by their classes to drowsy and awake lists
  for lb1, dr in enumerate(globals()[f"df{d}"]["epo"][0][0][1]):
    if lb1 in drowsy_inds[f"df{d}"]:
      # dr = lfilter(h, 1, dr)
      windows = librosa.util.frame(dr, frame_length=frame_length, hop_length=overlap).transpose(1, 0)
      han_windows = np.multiply(windows, np.hanning(frame_length))
      feats = []
      for win in han_windows:
        feats.append(np.array(np.abs(np.array(rfft(win)[10:180])).tolist()))
        # feats.append(mfccs_feats(win, 100))

      v_drowsy_freq.append(feats)

    if lb1 in awake_inds[f"df{d}"]:
      # dr = lfilter(h, 1, dr)
      windows = librosa.util.frame(dr, frame_length=frame_length, hop_length=overlap).transpose(1, 0)
      han_windows = np.multiply(windows, np.hanning(frame_length))
      feats = []
      for win in han_windows:
        feats.append(np.array(np.abs(np.array(rfft(win)[10:180])).tolist()))
        # feats.append(mfccs_feats(win, 100))

      v_awake_freq.append(feats)

In [10]:
v_awake_freq = pca.transform(np.real(np.array(v_awake_freq).reshape(len(v_awake_freq) * len(v_awake_freq[0]), len(v_awake_freq[0][0])))).reshape(len(v_awake_freq), len(v_awake_freq[0]), n_components).tolist()
v_drowsy_freq = pca.transform(np.real(np.array(v_drowsy_freq).reshape(len(v_drowsy_freq) * len(v_drowsy_freq[0]), len(v_drowsy_freq[0][0])))).reshape(len(v_drowsy_freq), len(v_drowsy_freq[0]), n_components).tolist()
v_awake_freq = scaler.transform(np.real(v_awake_freq).reshape(len(v_awake_freq) * len(v_awake_freq[0]), len(v_awake_freq[0][0]))).reshape(len(v_awake_freq), len(v_awake_freq[0]), len(v_awake_freq[0][0]))
v_drowsy_freq = scaler.transform(np.real(v_drowsy_freq).reshape(len(v_drowsy_freq) * len(v_drowsy_freq[0]), len(v_drowsy_freq[0][0]))).reshape(len(v_drowsy_freq), len(v_drowsy_freq[0]), len(v_drowsy_freq[0][0]))

In [11]:
v_d_label = [1] * (v_drowsy_freq.shape[0])
v_a_label = [0] * (v_awake_freq.shape[0])

# Zip the features and labels
v_data_d = list(zip(v_drowsy_freq, v_d_label))
v_data_a = list(zip(v_awake_freq, v_a_label))
v_data = v_data_a + v_data_d

# Shuffle the data
random.shuffle(v_data)

# Unzip the data
v_features, v_labels = zip(*v_data)

v_features = np.array(v_features, dtype=np.float16)
v_labels = np.array(v_labels, dtype=np.float16)

In [12]:
print(features.shape, labels.shape, v_features.shape, v_labels.shape)

(12964, 9, 7) (12964,) (1214, 9, 7) (1214,)


In [15]:
import tensorflow as tf
from tensorflow.keras import layers

input_shape = (features.shape[1], features.shape[2])

num_classes = 4

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="elu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def TransformerModel(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], input_shape[1], 1))(inputs)
    x = layers.Conv2D(filters=32, kernel_size=(3,3), activation='elu')(x)
    x = layers.MaxPool2D(pool_size=(2,2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3,1), activation='elu')(x)
    x = layers.MaxPool2D(pool_size=(1,2))(x)
    # x = layers.Conv2D(filters=32, kernel_size=(1,2), activation='elu')(x)
    # x = layers.MaxPool2D(pool_size=(1,2))(x)
    # x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128)(x)
    x = layers.Activation('elu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128)(x)
    x = layers.Activation('elu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128)(x)
    x = layers.Activation('elu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Reshape((1, x.shape[-1]))(x)
    x = TransformerEncoder(embed_dim=128, num_heads=4, ff_dim=128)(x)
    x = layers.GlobalMaxPooling1D()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = TransformerModel(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(clipnorm=1),
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [16]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

model_root = "/Users/albk/Documents/Code/Real_Projects/MicroSleep/model"
checkpoint = ModelCheckpoint(model_root + "/best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
# early_stop = EarlyStopping(monitor='val_accuracy', patience=3, mode='max', verbose=1)
history = model.fit(features, labels, epochs=10, validation_data=(v_features, v_labels), callbacks=[checkpoint])

Epoch 1/10
404/406 [============================>.] - ETA: 0s - loss: 0.3570 - accuracy: 0.8411
Epoch 1: val_accuracy improved from -inf to 0.53460, saving model to /Users/albk/Documents/Code/Real_Projects/MicroSleep/model/best_model.h5
406/406 [==============================] - 5s 10ms/step - loss: 0.3565 - accuracy: 0.8413 - val_loss: 0.9574 - val_accuracy: 0.5346
Epoch 2/10
403/406 [============================>.] - ETA: 0s - loss: 0.2687 - accuracy: 0.8906
Epoch 2: val_accuracy did not improve from 0.53460
406/406 [==============================] - 3s 7ms/step - loss: 0.2683 - accuracy: 0.8909 - val_loss: 1.2752 - val_accuracy: 0.4984
Epoch 3/10
402/406 [============================>.] - ETA: 0s - loss: 0.2336 - accuracy: 0.9066
Epoch 3: val_accuracy improved from 0.53460 to 0.54119, saving model to /Users/albk/Documents/Code/Real_Projects/MicroSleep/model/best_model.h5
406/406 [==============================] - 3s 8ms/step - loss: 0.2336 - accuracy: 0.9066 - val_loss: 1.1516 - val

In [ ]:
from keras.models import load_model

# Load the best model
best_model = load_model(model_root + "/best_model.h5")

# Evaluate the best model on the test set
test_loss, test_acc = best_model.evaluate(v_features, v_labels)
print('Test accuracy:', test_acc)

ValueError: Unknown layer: 'TransformerEncoder'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.